In [ ]:
%matplotlib inline

### Adjust edge masking in NGC6822 products

The default settings left obvious sidelobe structures in the final derived products. While these are relatively minor and only present in the broad mask, this notebook trims the outer edge by ~few arcmin to avoid any confusion on whether this structure is realy where the pb coverage lower.

In [ ]:
import matplotlib.pyplot as plt
from astropy.io import fits
from spectral_cube import SpectralCube
import scipy.ndimage as nd


from astropy.wcs.utils import proj_plane_pixel_scales

from pathlib import Path

In [ ]:
data_path = Path("/reduction/erickoch/LGLBS/line_imaging/derived/ngc6822")

clip_edge_width = 3 * u.arcmin

In [ ]:
# Make the edge clipping mask from finite coverage in one of the cubes.

cube = SpectralCube.read(data_path / "orig/ngc6822_C+D+tp_himidres.fits")

mid_chan = cube.shape[0] // 2

mask_orig = np.isfinite(cube.unitless_filled_data[mid_chan])

assert mask_orig.max() == True


plt.imshow(mask)

pix_scale = proj_plane_pixel_scales(cube.wcs.celestial)[0] * u.deg

clip_edge_width_pix = int(np.floor((clip_edge_width / pix_scale)))

print(clip_edge_width_pix)

mask = nd.binary_erosion(mask_orig, iterations=clip_edge_width_pix)

plt.contour(mask, levels=[0.5], colors='c')

print(mask.shape)

In [ ]:
# Compare this against the broad mask coverage:

# broad_mask = fits.open(data_path / "orig/ngc6822_C+D+tp_himidres_broadmask.fits")[0].data.sum(0)
broad_mask = fits.open(data_path / "orig/ngc6822_C+D+tp_hi21cm_0p8kms_broadmask.fits")[0].data.sum(0)

print(broad_mask.shape)

plt.imshow(broad_mask, origin='lower')
plt.contour(mask, levels=[0.5], colors='c')


In [ ]:
# Loop through all fits files and apply and cutout the new masked edge

from tqdm import tqdm

# Need to make separate masks per line type.
line_types = ['hilores', 'himidres', 'hi21cm_0p8kms']

for this_line in line_types:

    # Make the edge mask
    cube = SpectralCube.read(data_path / f"orig/ngc6822_C+D+tp_{this_line}.fits")

    mid_chan = cube.shape[0] // 2

    mask_orig = np.isfinite(cube.unitless_filled_data[mid_chan])

    assert mask_orig.max() == True

    pix_scale = proj_plane_pixel_scales(cube.wcs.celestial)[0] * u.deg

    clip_edge_width_pix = int(np.floor((clip_edge_width / pix_scale)))

    mask = nd.binary_erosion(mask_orig, iterations=clip_edge_width_pix)

    for this_fitsfile in tqdm(data_path.glob(f"*C+D+tp*{this_line}*.fits")):

        with fits.open(this_fitsfile, mode='update') as hdu:

            if len(hdu[0].shape) == 2:
                hdu[0].data[~mask] = np.NaN
                # hdu[0].data = hdu[0].data[nd.find_objects(mask)[0]]

            elif len(hdu[0].shape) == 3:

                # Loop over channels:
                for ii in range(hdu[0].shape[0]):
                    if "mask" in this_fitsfile.name:
                        hdu[0].data[ii][~mask] = 0
                    else:
                        hdu[0].data[ii][~mask] = np.NaN
                    # hdu[0].data[ii] = hdu[0].data[ii][nd.find_objects(mask)[0]]

            else:
                print(f"Unsure what to do with {this_fitsfile}")

            hdu.flush()

In [ ]:
hdu = fits.open(this_fitsfile)

hdu[0].data[0].shape

In [ ]:
this_fitsfile

In [ ]:
mask.shape